<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/Tweet_push.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import tweepy

# class MyStreamListener(tweepy.StreamListener):

#    def on_status(self, status):
#        print(status.text)

#    def on_error(self, status_code):
#        if status_code == 420:
#            #returning False in on_data disconnects the stream
#            return False

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth)

# myStreamListener = MyStreamListener()
# myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [0]:
# myStream.filter(languages=['english'])

In [2]:
!pip -q install pycurl oauth2 kafka-python

    100% |████████████████████████████████| 215kB 7.2MB/s 
    100% |████████████████████████████████| 266kB 22.3MB/s 


In [0]:
import time
import pycurl
import urllib
import json
import oauth2 as oauth

In [0]:
consumer_key  = "2wrYTQnh4poBbF5MsubR3sk4k"
consumer_secret = "93Yi0isNNgNPW4bFo8hsMpANUIw61lofgRRpz8H39P5V6o63tl"
access_token  = "2761651876-LoLkgE3EFYkqIHSGEPvzB3udtmATj4XNem2MemW"
access_token_secret = "1uDlUpdk6mrinFKFl3vN06xIVeORaa60EjubXrEOSpYKc"

In [0]:
API_ENDPOINT_URL = 'https://stream.twitter.com/1.1/statuses/filter.json'
USER_AGENT = 'TwitterStream 1.0' # This can be anything really

In [6]:
# You need to replace these with your own values
OAUTH_KEYS = {'consumer_key': consumer_key,
              'consumer_secret': consumer_secret,
              'access_token_key': access_token,
              'access_token_secret': access_token_secret}
print(OAUTH_KEYS)

{'consumer_secret': '93Yi0isNNgNPW4bFo8hsMpANUIw61lofgRRpz8H39P5V6o63tl', 'access_token_key': '2761651876-LoLkgE3EFYkqIHSGEPvzB3udtmATj4XNem2MemW', 'consumer_key': '2wrYTQnh4poBbF5MsubR3sk4k', 'access_token_secret': '1uDlUpdk6mrinFKFl3vN06xIVeORaa60EjubXrEOSpYKc'}


In [0]:
POST_PARAMS = {'include_entities': 0,
               'stall_warning': 'true',
               'track': 'iphone,ipad,ipod',
               'languages':'en',
              'location': '-74, 40, -73, 41'}

In [0]:
class TwitterStream:
    def __init__(self, timeout=False):
        self.oauth_token = oauth.Token(key=OAUTH_KEYS['access_token_key'], secret=OAUTH_KEYS['access_token_secret'])
        self.oauth_consumer = oauth.Consumer(key=OAUTH_KEYS['consumer_key'], secret=OAUTH_KEYS['consumer_secret'])
        self.conn = None
        self.buffer = ''
        self.timeout = timeout
        self.setup_connection()

    def setup_connection(self):
        """ Create persistant HTTP connection to Streaming API endpoint using cURL.
        """
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        # Restart connection if less than 1 byte/s is received during "timeout" seconds
        if isinstance(self.timeout, int):
            self.conn.setopt(pycurl.LOW_SPEED_LIMIT, 1)
            self.conn.setopt(pycurl.LOW_SPEED_TIME, self.timeout)
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL)
        self.conn.setopt(pycurl.USERAGENT, USER_AGENT)
        # Using gzip is optional but saves us bandwidth.
        self.conn.setopt(pycurl.ENCODING, 'deflate, gzip')
        self.conn.setopt(pycurl.POST, 1)
        self.conn.setopt(pycurl.POSTFIELDS, urllib.urlencode(POST_PARAMS))
        self.conn.setopt(pycurl.HTTPHEADER, ['Host: stream.twitter.com',
                                             'Authorization: %s' % self.get_oauth_header()])
        # self.handle_tweet is the method that are called when new tweets arrive
        self.conn.setopt(pycurl.WRITEFUNCTION, self.handle_tweet)

    def get_oauth_header(self):
        """ Create and return OAuth header.
        """
        params = {'oauth_version': '1.0',
                  'oauth_nonce': oauth.generate_nonce(),
                  'oauth_timestamp': str(int(time.time()))}
        req = oauth.Request(method='POST', parameters=params, url='%s?%s' % (API_ENDPOINT_URL,
                                                                             urllib.urlencode(POST_PARAMS)))
        req.sign_request(oauth.SignatureMethod_HMAC_SHA1(), self.oauth_consumer, self.oauth_token)
        
       
        return str(req.to_header()['Authorization']).encode('utf-8')

    def start(self):
        """ Start listening to Streaming endpoint.
        Handle exceptions according to Twitter's recommendations.
        """
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                # Network error, use linear back off up to 16 seconds
                print('Network error: %s' % self.conn.errstr())
                print('Waiting %s seconds before trying again' % backoff_network_error)
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)
            if sc == 420:
                # Rate limit, use exponential back off starting with 1 minute and double each attempt
                print('Rate limit, waiting %s seconds' % backoff_rate_limit)
                time.sleep(backoff_rate_limit)
                backoff_rate_limit *= 2
            else:
                # HTTP error, use exponential back off up to 320 seconds
                print('HTTP error %s, %s' % (sc, self.conn.errstr()))
                print('Waiting %s seconds' % backoff_http_error)
                time.sleep(backoff_http_error)
                backoff_http_error = min(backoff_http_error * 2, 320)

    def handle_tweet(self, data):
        """ This method is called when data is received through Streaming endpoint.
        """
        self.buffer += str(data)
        if data.endswith(b'\r\n') and self.buffer.strip():
            # complete message received
            message = json.loads(self.buffer)
            self.buffer = ''
            msg = ''
            if message.get('limit'):
                print('Rate limiting caused us to miss %s tweets' % (message['limit'].get('track')))
            elif message.get('disconnect'):
                raise Exception('Got disconnect: %s' % message['disconnect'].get('reason'))
            elif message.get('warning'):
                print('Got warning: %s' % message['warning'].get('message'))
            else:
                print(message)
                print('Got tweet with text: %s' % message.get('text'))
            
            producer.send('tweetv1', value=message) 

In [0]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
from json import dumps

In [0]:
producer = KafkaProducer(bootstrap_servers=['18.216.34.97:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [0]:
ts = TwitterStream()
ts.setup_connection()
ts.start()

{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u"\u30e2\u30a4!iPhone\u304b\u3089\u914d\u4fe1\u4e2d\uff01\n\u7121\u8a00\u3067\u306e\u6f5c\u308a\u306f\u7981\u6b62\u3067\u3059\n\u305f\u3063\u3064\u3093\u4e3b\u3057\u3066\u307e\u3059( '\u03c9')\n https://t.co/tzcXl0Fk2o", u'is_quote_status': False, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 1120023446828617731, u'favorite_count': 0, u'source': u'<a href="http://twitcasting.tv/" rel="nofollow">TwitCasting</a>', u'retweeted': False, u'coordinates': None, u'timestamp_ms': u'1555869373410', u'entities': {u'user_mentions': [], u'symbols': [], u'hashtags': [], u'urls': [{u'url': u'https://t.co/tzcXl0Fk2o', u'indices': [45, 68], u'expanded_url': u'https://cas.st/202db2cf', u'display_url': u'cas.st/202db2cf'}]}, u'in_reply_to_screen_name': None, u'id_str': u'1120023446828617731', u'retweet_count': 0, u'in_reply_to_user_id': None, u'favorited': False, u'user': {u'follow_request_sent': None, u'profile_use_back

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 3088)
Waiting 0.25 seconds before trying again
{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u'RT @HOMOSEXUALIZING: 6th grade me coming home and opening my 1st gen ipod touch https://t.co/suOvFcGuDf', u'is_quote_status': False, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 1120023479854592000, u'favorite_count': 0, u'entities': {u'user_mentions': [{u'id': 440467055, u'indices': [3, 19], u'id_str': u'440467055', u'screen_name': u'HOMOSEXUALIZING', u'name': u'jack'}], u'symbols': [], u'hashtags': [], u'urls': [], u'media': [{u'source_user_id': 440467055, u'source_status_id_str': u'1119472573304193024', u'expanded_url': u'https://twitter.com/HOMOSEXUALIZING/status/1119472573304193024/photo/1', u'display_url': u'pic.twitter.com/suOvFcGuDf', u'url': u'https://t.co/suOvFcGuDf', u'media_url_https': u'https://pbs.twimg.com/media/D4kqXsJW0AEbMSe.jpg', u'source_user_id_str': u'440467055', u'source_status_id': 1119472

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 7328)
Waiting 1.25 seconds before trying again
{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u'RT @HOMOSEXUALIZING: 6th grade me coming home and opening my 1st gen ipod touch https://t.co/suOvFcGuDf', u'is_quote_status': False, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 1120023772818440192, u'favorite_count': 0, u'entities': {u'user_mentions': [{u'id': 440467055, u'indices': [3, 19], u'id_str': u'440467055', u'screen_name': u'HOMOSEXUALIZING', u'name': u'jack'}], u'symbols': [], u'hashtags': [], u'urls': [], u'media': [{u'source_user_id': 440467055, u'source_status_id_str': u'1119472573304193024', u'expanded_url': u'https://twitter.com/HOMOSEXUALIZING/status/1119472573304193024/photo/1', u'display_url': u'pic.twitter.com/suOvFcGuDf', u'url': u'https://t.co/suOvFcGuDf', u'media_url_https': u'https://pbs.twimg.com/media/D4kqXsJW0AEbMSe.jpg', u'source_user_id_str': u'440467055', u'source_status_id': 1119472

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 7288)
Waiting 2.25 seconds before trying again
{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u'\u30e2\u30a4\uff01iPhone\u304b\u3089\u30ad\u30e3\u30b9\u914d\u4fe1\u4e2d -\u3053\u3044\u3084 https://t.co/njYDrHnv7o', u'is_quote_status': False, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 1120023785921318912, u'favorite_count': 0, u'source': u'<a href="http://twitcasting.tv/" rel="nofollow">TwitCasting</a>', u'retweeted': False, u'coordinates': None, u'timestamp_ms': u'1555869454256', u'entities': {u'user_mentions': [], u'symbols': [], u'hashtags': [], u'urls': [{u'url': u'https://t.co/njYDrHnv7o', u'indices': [23, 46], u'expanded_url': u'https://cas.st/202db349', u'display_url': u'cas.st/202db349'}]}, u'in_reply_to_screen_name': None, u'id_str': u'1120023785921318912', u'retweet_count': 0, u'in_reply_to_user_id': None, u'favorited': False, u'user': {u'follow_request_sent': None, u'profile_use_background_image

In [0]:
producer.close()

In [0]:
from kafka import KafkaConsumer
from json import loads

In [0]:
consumer = KafkaConsumer(
    'numtest',
     bootstrap_servers=['18.216.34.97:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group310',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [0]:
for message in consumer:
    message = message.value
    print(message)
    print(type(message))